Water Data: RFECV, grid search superparameter, ten fold cross validation


水体数据: RFECV, 网格搜索超参，十折交叉验证

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import myfunction as mf
import myrfecv as mr
path_data_raw = "C:/Users/dell/OneDrive/file/"
path_country_nc = "C:/Users/dell/OneDrive/file/nc"
path_onedrive_csv = "C:/Users/dell/OneDrive/file/csv/"
path_one_spdb = 'C:/Users/dell/OneDrive/file/SPDB/'
drive_letter = 'E:'

path_pre = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/pretreatment/"
path_match = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/match/"
path_semdata = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/semdata/"

path_2_preanalysis_data = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part2_sem/preanalysis/"
path_2_preanalysis_fig = drive_letter + "/wyy/code_project/running_outcome/final_fig/SPDB/part2_sem/preanalysis/"
path_3_sw_forecast = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/"
path_temp = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/temp/"


mark_num = "25"
meta_name = "meta_data.csv"
list_pfas =['PFOA', 'PFNA', 'PFDA', 'PFUnDA','PFDoDA','PFTrDA', 'PFTeDA', 'PFHxS', 'PFOS', 'FOSA', 'PFBA', 'PFPeA', 'PFHxA', 'PFHpA','PFBS']
list_pfas_lc = ['PFOA', 'PFNA', 'PFDA', 'PFUnDA','PFDoDA','PFTrDA', 'PFTeDA', 'PFHxS', 'PFOS', 'FOSA']
list_pfas_sc = ['PFBA', 'PFPeA', 'PFHxA', 'PFHpA','PFBS']
list_color = ["#4d8cbf", "#4f9c8b", "#555c6c", "#d77563", "#7d84a8", "#84aeb8", "#c3473b", "#89756d","#ffb3cc","#9a7ebf","#ffddb8", "#c4eaff", "#d1c6ff", "#c2ffbf", "#f5f5b0"]
dic_color = dict(zip(list_pfas,list_color))

### RFECV+超参寻找

In [ ]:
# RFECV

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
df_sw_data_raw = pd.read_csv(path_semdata + "sem_sw_s7_au_to_"+mark_num+"_avg.csv")
df_sw_data_raw['year'] = (df_sw_data_raw['year'] - 2000) / (2020 - 2000)
columns_to_drop = ['lon_grid', 'lat_grid']
df_sw_data = df_sw_data_raw.drop(columns=columns_to_drop)

print(df_sw_data.columns)

path_sw_rfecv_data = drive_letter + '/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/'
path_sw_rfecv_fig = drive_letter + '/wyy/code_project/running_outcome/final_fig/SPDB/part3_forecast/sw_forecast/'

dict_clf = {
    'GBDT': GradientBoostingRegressor(random_state=202406),
    'RF': RandomForestRegressor(random_state=202406),
}

str_marker_type = 'sws7'
for key, clf in dict_clf.items():
    mr.start_rfecv(key, clf, df_sw_data, str_marker_type, path_sw_rfecv_data, path_sw_rfecv_fig, 'cv')
# 58m

df_sw_rf_rfecv = pd.read_csv(path_sw_rfecv_data + str_marker_type + '_rfecv_r2_RFcv.csv')
df_sw_rf_rfecv['model'] = 'RF'
df_sw_gbdt_rfecv = pd.read_csv(path_sw_rfecv_data + str_marker_type + '_rfecv_r2_GBDTcv.csv')
df_sw_gbdt_rfecv['model'] = 'GBDT'

df_sw_rfecv = pd.concat([df_sw_rf_rfecv,df_sw_gbdt_rfecv],axis=0)
df_sw_rfecv = df_sw_rfecv.sort_values(by='mean_test_score', ascending=False)
print(df_sw_rfecv.head())
max_index = df_sw_rfecv['mean_test_score'].idxmax()
min_features = df_sw_rfecv.loc[max_index, 'min_features'].tolist()[0]
max_model = df_sw_rfecv.loc[max_index, 'model'].tolist()[0]
max_score = round(df_sw_rfecv.loc[max_index, 'mean_test_score'].tolist()[0],3)
df_sw_rfecv.to_csv(path_sw_rfecv_data + str_marker_type + '_merge_rfecv.csv',index=False)
print(f'model:{max_model} min_feature:{min_features} score:{max_score}')

# 123min

In [ ]:

scoring = 'r2'
str_describe = 'sws7'
df_sw_data_raw = pd.read_csv(path_semdata + 'sem_sw_s7_au_to_' + mark_num +'_avg.csv')
df_sw_data_raw['year'] = (df_sw_data_raw['year'] - 2000) / (2020 - 2000)
columns_to_drop = ['lon_grid', 'lat_grid']
df_sw_sem = df_sw_data_raw.drop(columns=columns_to_drop)

path_sw_rfecv_data = drive_letter + '/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/'

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 3, 5, 10],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
}
param_grid_gbdt = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1],
    'max_depth': [None, 3, 5, 10],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
}
cv_result = mr.grid_search_param(df_sw_sem, str_describe, scoring, path_sw_rfecv_data, param_grid_rf, param_grid_gbdt, 'cv')
cv_result.to_csv(path_sw_rfecv_data + 'ml_cv_results.csv', index=False)
# 206min

In [ ]:
cv_result = pd.read_csv(path_sw_rfecv_data + 'ml_cv_results.csv')
results = mr.get_tree_best_model(cv_result)
results.to_csv(path_sw_rfecv_data + 'ml_cv_best.csv', index=False)
results = pd.read_csv(path_sw_rfecv_data + 'ml_cv_best.csv')
for index, value in results['param_max_depth'].items():
    if pd.isna(value) or value == '':
        results.at[index, 'param_max_depth'] = 'None'
results.to_csv(path_sw_rfecv_data + 'ml_cv_best.csv', index=False)

In [6]:

scoring = 'r2'
str_describe = 'sws7'

df_sw_data_raw = pd.read_csv(path_semdata + 'sem_sw_s7_au_to_' + mark_num +'_avg.csv')
df_sw_data_raw['year'] = (df_sw_data_raw['year'] - 2000) / (2020 - 2000)
columns_to_drop = ['lon_grid', 'lat_grid']
df_sw_sem = df_sw_data_raw.drop(columns=columns_to_drop)

path_sw_rfecv_data = drive_letter + '/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/'
selected_features_gbdt = pd.read_csv(path_sw_rfecv_data + str_describe + "_rfecv_features_GBDTcv.csv")
selected_features_gbdt = selected_features_gbdt[selected_features_gbdt["Rank"] == 1]["Feature"].values

selected_features_rf = pd.read_csv(path_sw_rfecv_data + str_describe + "_rfecv_features_RFcv.csv")
selected_features_rf = selected_features_rf[selected_features_rf["Rank"] == 1]["Feature"].values

best_params = pd.read_csv(path_sw_rfecv_data + 'ml_cv_best.csv')

results  = mr.tree_inf(df_sw_sem, selected_features_gbdt, selected_features_rf, best_params)
print("GBDT R2 scores for each fold:", results['gbdt_r2'])
print("GBDT MSE scores for each fold:", results['gbdt_mse'])
print("RF R2 scores for each fold:", results['rf_r2'])
print("RF MSE scores for each fold:", results['rf_mse'])

Average R2 for GBDT: 0.8487539106255861
Average MSE for GBDT: 0.15017637657050928
Average R2 for RF: 0.8252560967221937
Average MSE for RF: 0.17364791440968308
GBDT R2 scores for each fold: [0.8378904244833526, 0.8625286169718952, 0.8563917076422805, 0.8444044451687451, 0.840798367604384, 0.8199255935801109, 0.8665889917318368, 0.8598529008523887, 0.8444275274915376, 0.8547305307293285]
GBDT MSE scores for each fold: [0.1576541308861602, 0.13667100849275707, 0.14953307329439078, 0.1547260787257209, 0.15289248078701875, 0.17554624018696394, 0.13626279275553266, 0.1388817055496256, 0.15259676354415366, 0.1469994914827694]
RF R2 scores for each fold: [0.8130109187409805, 0.8256468659493262, 0.8221251389032479, 0.829933824616413, 0.8329525672831832, 0.7962265889477225, 0.8370237421716779, 0.8345431555910081, 0.8235827498366645, 0.8375154151817131]
RF MSE scores for each fold: [0.18184984444712812, 0.17333802962982345, 0.18521336202059807, 0.16911583669289665, 0.16042735248913206, 0.1986493

In [ ]:

scoring = 'r2'
str_describe = 'sws7'
df_sw_sem = pd.read_csv(path_semdata + 'sem_sw_s7_au_to_' + mark_num +'_avg.csv')
df_sw_sem['year'] = (df_sw_sem['year'] - 2000) / (2020 - 2000)
path_sw_rfecv_data = drive_letter + '/wyy/code_project/running_outcome/final_data/SPDB/part3_forecast/sw_forecast/'


best_params = pd.read_csv(path_sw_rfecv_data + 'ml_cv_best.csv')
best_params = best_params.sort_values(by='mean_test_score', ascending=False)

selected_features = pd.read_csv(path_sw_rfecv_data + str_describe + "_rfecv_features_RFcv.csv")
selected_features = selected_features[selected_features["Rank"] == 1]["Feature"].values
print(len(selected_features))
# Get best parameters for GBDT and RF
model_params = best_params[best_params["model"] == "RF"].iloc[0]
# Define models with best parameters    
if model_params["param_max_depth"] == 'None':
    param_max_depth = None
else:
    param_max_depth = float(model_params["param_max_depth"])
    param_max_depth = int(param_max_depth)
select_model = RandomForestRegressor(
    max_depth=param_max_depth,
    min_samples_leaf=int(model_params["param_min_samples_leaf"]),
    min_samples_split=int(model_params["param_min_samples_split"]),
    n_estimators=int(model_params["param_n_estimators"])
)
# Prepare data
X_train = df_sw_sem[selected_features]
y_train = df_sw_sem['value']

select_model.fit(X_train, y_train)


In [ ]:
df_data_raw = pd.read_csv(path_match + 'lr_fish_avg.csv')
df_data_raw = df_data_raw[(df_data_raw['length_last'].notna())&(df_data_raw['weight_last'].notna())&(df_data_raw['troph_last'].notna())]
df_data_raw = mf.id_to_str(df_data_raw,['posname'])
df_data_raw.to_csv(path_match + 'lr_fish_full.csv')


In [8]:

df_forecast_data = pd.read_csv(path_semdata + 'sem_lr_fish_au_to_25_avg.csv')
df_forecast_data['year_raw'] = df_forecast_data['year']
df_forecast_data['year'] = (df_forecast_data['year'] - 2000) / (2020 - 2000)

X_forecast = df_forecast_data[selected_features]
y_pred = select_model.predict(X_forecast)
df_forecast_data['sw_value'] = y_pred
print(df_forecast_data.columns)
df_forecast_data.drop(columns=['year'], inplace=True)
df_forecast_data = df_forecast_data.rename(columns={'year_raw':'year'})

df_forecast_data.to_csv(path_semdata + 'sem_lrsws7_au_to_25_avg_full.csv',index=False)


Index(['TOTALS_CO2_E', 'GDP', 'u10', 'cth', 'C2F6', 'licd', 'night_lights',
       'ref_ice', 'cfc', 'ltlt', 'stl1', 'cl', 'v10', 'v100',
       'estuary_distance', 'TNR_Ship_CO2_E', 'sp', 'sshf', 'tp',
       'port_distance', 'human_footprint', 'HFC', 'ctt', 'HCFC', 'ref_liq',
       'skt', 'mine_distance', 'ctp', 'fluorite_consumption', 'sro', 'tcrw',
       'paper_consumption', 'swvl1', 'tcsw', 'SF6', 'nships_smoothed',
       'population', 'CF4', 'i10fg', 'global_salinity', 'SWD_INC_CO2_E',
       'ssro', 'si10', 'wrap_consumption', 'e', 'lict', 'ssr', 'str', 'z',
       'u100', 'dl', 'lon_grid', 'lat_grid', 'year', 'log_Koc', 'solubility',
       'sp_length', 'po_carbon', 'melting_point', 'log_Kow', 'log_Koa_wet',
       'boiling_point', 'sp_weight', 'log_Kaw', 'density', 'po_f_carbon',
       'sp_troph', 'log_Koa_dry', 'po_xlogp', 'log_KHxd_air', 'log_Koil_w',
       'po_m_w', 'log_Koil_air', 'value', 'organ_muscle', 'organ_whole',
       'habitat', 'po_chain', 'year_raw', 'sw_va